In [235]:
from sklearn.datasets import load_boston
import numpy as np

x = load_boston().data
# row_sums = x.sum(axis=1)
# x = x / row_sums[:, np.newaxis]
# x = np.c_[np.ones(x.shape[0]), x]
y = load_boston().target

In [236]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=29)

x_train.shape

(379, 13)

In [63]:
def scal(x):
    nx = np.array(x.shape)
    for c in range(x.shape[1]):
        for r in range(x.shape[0]):
            nx[r][c] = 1/x[r][c]
    return nx

tes = np.array([[1, 30, 2, 37], [1, 38, 2, 37]])

row_sums = tes.sum(axis=1)
new_matrix = tes / row_sums[:, np.newaxis]

new_matrix

array([[ 0.01428571,  0.42857143,  0.02857143,  0.52857143],
       [ 0.01282051,  0.48717949,  0.02564103,  0.47435897]])

In [64]:
def polynom(m, deg):
    n_rows, n_cols = m.shape[0], m.shape[1]
    nm = np.zeros((m.shape[0],n_cols*deg))
    for d in range(deg):
        for c in range(n_cols):
            for r in range(n_rows):
                nm[r][n_cols*d+c] = m[r][c]**(d+1)
    return nm

In [164]:
def normalEq(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

theta = normalEq(x_train, y_train)

print(theta)

preds = x_test @ theta

[  3.59818983e+01  -9.35811334e-02   5.31800698e-02   3.67125390e-02
   3.43338847e+00  -1.48087291e+01   3.54024870e+00  -2.54930954e-03
  -1.36624878e+00   3.22244933e-01  -1.29376404e-02  -9.65171087e-01
   1.10630023e-02  -5.51213763e-01]


$X^{'}X=\begin{bmatrix} 7 & 38.5\\  38.5& 218.75 \end{bmatrix}$

In [334]:
def scaling(X):
    l = []
    for k in range(X.shape[1]):
        x = X[:,k]
        def f(e):
            tmp0 = sum(x) / len(x)
            tmp1 = max(x) - min(x)
            return (e - tmp0) / tmp1
        m =  list(map(float, (list(map(f, x)))))
        l.append(m)
    return np.matrix(l).transpose()

In [360]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), X]
    print(X.shape)
    theta = np.random.rand(1, X.shape[1])
    print(theta.T.shape)
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = ((X * theta.T) - y.T).T * nX
            theta[:,c] = nT - eta * gradients
    return theta

yt = np.array([y_train.T])
xt = scaling(x_train)

theta = BGD(xt, yt, 1000, 0.001)

print(theta)

(379, 14)
(14, 1)
[[ 22.61794195  -6.04593092   5.02942447   0.9046596    3.47654907
   -6.83273622  18.72882357  -0.20764987 -14.35065403   7.01719818
   -6.42739183  -9.04035945   4.46536347 -20.05722522]]


In [338]:
def MGD(A, y, eta, d):
    X = np.c_[np.ones(len(A)), A]
    theta = np.random.rand(1, X.shape[1])
    m = len(X)
    delta = cost(X, y, theta)
    steps = 0
    while delta > d:
        beg = cost(X, y, theta)
        for j in range(X.shape[1]):
            hyp = X * theta.T
            diff = hyp - y.T
            grad = diff.T * X[:,j]
            theta[:,j] = theta[:,j] - eta * grad
        end = cost(X, y, theta)
        delta = abs(end - beg)
        steps+=1
    return (theta, steps)

def cost(A, y, theta):
    return (1 / len(A)) * float(sum([x * x for x in A * theta.T - y.T]))

yt = np.array([y_train.T])
xt = scaling(x_train)

theta, steps = MGD(xt, yt, 0.001, 0.001)

print(steps)

458


In [355]:
preds = np.c_[np.ones(len(x_test)), scaling(x_test)] @ theta.T

preds

matrix([[ 24.76444322],
        [ 41.15559461],
        [ 21.93852511],
        [ 18.23271052],
        [ 19.27594757],
        [ 21.13576644],
        [ 28.99675252],
        [ 32.25149197],
        [ 18.72633136],
        [  5.8757972 ],
        [ 43.58878742],
        [ 27.77584277],
        [ 39.78589125],
        [ 13.84579933],
        [ 24.59846886],
        [ 28.05859242],
        [ 23.89191744],
        [ 26.0721882 ],
        [ 18.95919159],
        [ 20.46077954],
        [ 38.22442717],
        [ 21.39827615],
        [ 19.39726721],
        [ 20.93762871],
        [ 33.6700485 ],
        [ 21.00910764],
        [ 35.58040407],
        [ 25.16053918],
        [ 29.38320877],
        [ 34.2710812 ],
        [ 33.90797227],
        [ 26.06723928],
        [ 15.00812784],
        [ 26.13773878],
        [ 15.06411414],
        [  6.70235555],
        [ 23.3702366 ],
        [  7.68000001],
        [ 18.39459113],
        [ 20.60022097],
        [ 28.34877159],
        [ 24.131

In [163]:
def SGD(X, y, eta, n_epoch):
    theta = np.random.rand(X.shape[1], 1)
    m = X.shape[0]
    for epoch in range(n_epoch):
        for c in range(X.shape[1]):
            for i in range(m):
                ri = np.random.randint(m)
                guess = X[ri:ri+1, c:c+1] @ theta[c:c+1]
                diff = guess - y[ri:ri+1]
                grad = X[ri:ri+1, c:c+1].T @ diff
                theta[c:c+1] = theta[c:c+1] - grad * eta
    print(diff)
    print(grad)
    print(theta)

SGD(x_train, y_train, 0.1, 50)

/Users/kkgh070/anaconda3/envs/porto/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in subtract
  # This is added back by InteractiveShellApp.init_path()


[[ nan]]
[[ nan]]
[[ 23.85098248]
 [         nan]
 [         nan]
 [         nan]
 [ 29.86148861]
 [ 36.5634841 ]
 [         nan]
 [         nan]
 [  8.52494419]
 [         nan]
 [         nan]
 [         nan]
 [         nan]
 [         nan]]


In [67]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = LinearRegression()
polynomial_regression = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
polynomial_regression.fit(x_train, y_train)
preds = polynomial_regression.predict(x_test)

In [68]:
m = LinearRegression()

m.fit(polynom(x_train, 2), y_train)

preds = m.predict(polynom(x_test, 2))

print('weights: ')
print(m.coef_)
print('Intercept: ')
print(m.intercept_)

weights: 
[ -4.82353170e-01  -6.11811058e-02  -1.03006200e-01   1.52751036e+00
  -2.66093614e+01  -1.72065010e+01  -5.67860772e-02  -2.20943742e+00
   5.80765699e-01  -2.01868784e-02  -6.10801742e+00   2.14283984e-02
  -1.54975811e+00   4.96884001e-03   8.07538384e-04   6.18487450e-03
   1.52751037e+00   3.81582232e+00   1.54888132e+00   5.40426439e-04
   1.03788287e-01  -4.23190591e-03   6.78223968e-06   1.49536968e-01
  -3.09227294e-05   2.80283423e-02]
Intercept: 
164.391664059


In [356]:
from sklearn.metrics import explained_variance_score

explained_variance_score(y_test, preds)

0.76066541110391184